In [3]:
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

## What is in Ackerman & Marley 01

The paper currently states that for reynolds numbers between 1-1000, it takes values from Table 10-1 in Pruppachar & Klett, and fits the line y = 0.8*x - 0.01*x^2, where y = log(N_re) and x = log(cd * N_re^2). 

Below, is the data from Table 10-1 with the fitted line used in the code

In [120]:
f = figure(width=400, height=400,y_axis_label = 'log(R_N)', x_axis_label='log (C_d * R_N^2)')
def x(r,c):
    return np.log(c*r**2)

def y(r):
    return np.log(r)

#information from Table 10-1
r = np.array([1,10,30,57,100,300,1000])
cd = np.array([24, 4.29, 2.11,1.51,1.1,0.63,.45])
f.circle(x(r,cd), y(r),size=5, color='black',legend_label='P&K Table 10-1')
xx = np.linspace(0,13,10)
#this is the difference. A&M defines their "X" with that deivide by 24 
x1 = np.log(np.exp(xx)/24)
yy = 0.8*(x1) - 0.01*(x1)**2 

b0,b1,b2,b3,b4,b5,b6 = -0.318657e1, 0.992696, -.153193e-2, -.987059e-3, -.578878e-3, 0.855176e-4, -0.327815e-5
yy1 = b0 + b1*xx**1 + b2*xx**2 + b3*xx**3 + b4*xx**4 + b5*xx**5 + b6*xx**6

f.line(xx,yy,color='blue',legend_label='A&M01 Regime 2')
f.line(xx,yy1,color='red',legend_label='P & K Regime 2')
f.line(np.log(0.45*np.exp(np.linspace(np.log(300),10,10))**2.0),np.linspace(np.log(300),10,10),color='green',legend_label='Asymtotic Regime 3')
f.legend.location='bottom_right'
show(f)

## Computing MMW from chemeq

In [47]:
import pickle as pk 

chem = pk.load(open('/Users/nbatalh1/Documents/codes/PICASO/picaso/reference/chem_full.pic','rb'), encoding='latin1')

logCtoO, logMet, Tarr, logParr, gases=chem
H2Oarr, CH4arr, COarr, CO2arr, NH3arr, N2arr, HCNarr, H2Sarr,PH3arr, C2H2arr, C2H6arr, Naarr, Karr, TiOarr, VOarr, FeHarr, Harr,H2arr, Hearr, mmw = gases

ValueError: too many values to unpack (expected 20)

In [66]:
ngas = gases.shape[3]

In [72]:
tointerp = gases[:,:,:,-1,:]
tointerp.shape

(22, 25, 36, 95)

In [78]:
fig = figure(x_axis_type='log',height=200,width=400)
for i in range(tointerp.shape[0]): # C/O
    fig.line( logMet, tointerp[i,:,0,0] ,color='red')
for i in range(tointerp.shape[2]): # T
    fig.line( logMet, tointerp[0,:,i,0] ,color='blue')
for i in range(tointerp.shape[3]): # Pressure
    fig.line( logMet, tointerp[0,:,0,i] ,color='green')
show(fig)

# Inverting PVAPs automatically 

Trying to figure how invert this function:

$P_{vap}(T) = P_{sat}$ 

where $P_{va}$ is some undescribed function of temperature. and $P_{sat}$ is linear with pressure. 

Want this: 

$T_{vap}(P) = ...$



In [81]:
import gas_properties 
import pvaps

In [87]:
type(pvaps.Cr)

function

In [110]:
mh = 1
mmw = 2.2
pressure = np.logspace(-3, 2, 20)
a, gas_mmr ,a = gas_properties.Cr(mmw,mh=mh)
pvap_cr = pvaps.Cr(500, mh=np.log10(1)) #returned in bars 

def find_t(t_test, p_test = None, mh=None, mmw=None, pvap_fun = None,gas_p_fun=None):    
    """
    Parameters
    ----------
    t_test : float 
        Temp (K)
    p_test : float 
        Pressure bars 
    mh : float 
        NON log mh .. aka MH=1 for solar 
    mmw : float 
        mean molecular weight (2.2 for solar)
    pvap_fun : function
        Vapor function for a particular gas from `pvap`
    gas_p_fun : function
        Gas Property function for a particular gas from `gas_properties`
    """
    #get gas mixing ratio 
    gas_mw, gas_mmr ,rho = gas_p_fun(mmw,mh=mh)
    #get vapor pressure
    pv = gas_mw/mmw*pvap_fun(t_test, mh=np.log10(mh))/1e6 #dynes to bars 
    #get p_saturation
    psat = gas_mmr*p_test*mh 
    return np.log10(pv) - np.log10(psat)
    


In [111]:
from scipy import optimize

gas_name = 'Cr'
pvap_fun = getattr(pvaps, gas_name)
gas_p_fun = getattr(gas_properties, gas_name)

 
temps = []
for p in pressure: 
    temp = optimize.root_scalar(find_t, 
                    bracket=[50, 3000], method='brentq', 
                    args=(p, mh, mmw, pvap_fun ,gas_p_fun))
    temps += [temp.root]

/Users/nbatalh1/.conda/envs/picaso/lib/python3.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in log10


In [112]:
import condensables

In [113]:
t,p = condensables.Cr()
f = figure(y_axis_type='log',y_range=[1e2,1e-3], height=400)
f.line(t,p,legend_label='Morley',color='red')
f.line(temps, pressure)
show(f)

In [148]:
qscat = np.reshape(np.linspace(1,12,12),(3,4)).T
nradii = 3
radius = np.zeros(3)
nwave = 4 


SyntaxError: invalid syntax (<ipython-input-149-5d3a3d86d1c1>, line 3)

In [151]:
sum([[' ']+list(i) for i in qscat.T],[])

[' ', 1.0, 2.0, 3.0, 4.0, ' ', 5.0, 6.0, 7.0, 8.0, ' ', 9.0, 10.0, 11.0, 12.0]